

#EXAM SCHEDULE:

##LAB-3

###Scenario Description:
Courses as nodes: Each node in the graph represents a course.
Edges represent conflicts: An edge between two courses means there are students enrolled in both, so these courses cannot be scheduled at the same time.
Colors as time slots: Each color represents a unique time slot. Our goal is to assign each course to a time slot such that no two conflicting courses are assigned the same time slot.
Modifications to the Code:
Graph Construction: Instead of generating a random graph, the graph should represent the actual conflict between courses. We'll manually create a graph where edges represent conflicts between courses.
Node Labels: Nodes will be labeled with course names instead of numbers.
Color Representation: Colors will represent time slots, and they will be printed as "Time Slot 1", "Time Slot 2", etc.
I'll walk through the changes required:

###1. Graph Representation of Courses
I will replace the random graph generation with a manually created graph representing courses with conflicts.

###2. Heuristic Selection
I'll allow the user to choose between Minimum Remaining Values (MRV) and Degree Heuristic. These heuristics help decide which course should be scheduled next

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Step 1: Generate valid colors (time slots) using HEX codes
def generate_colors(num_nodes):
    color_mapping = {
        'Time Slot 1': '#FF5733',  # Red
        'Time Slot 2': '#33FF57',  # Green
        'Time Slot 3': '#3357FF',  # Blue
        'Time Slot 4': '#FFFF33',  # Yellow
        'Time Slot 5': '#FF33FF'   # Magenta
    }
    time_slots = list(color_mapping.keys())[:min(num_nodes, len(color_mapping))]
    return time_slots, color_mapping

# Step 2: Manually create a course conflict graph
def create_exam_schedule_graph():
    G = nx.Graph()
    courses = ['Math', 'Physics', 'Chemistry', 'Biology', 'English', 'History', 'Computer Science']

    # Add courses (nodes)
    G.add_nodes_from(courses)

    # Add conflicts (edges) between courses with common students
    conflicts = [('Math', 'Physics'), ('Math', 'Computer Science'), ('Physics', 'Chemistry'),
                 ('Biology', 'Chemistry'), ('English', 'History'), ('History', 'Computer Science')]
    G.add_edges_from(conflicts)

    return G

# Step 3: Visualize the exam schedule graph
def visualize_graph(G, assignment, step, pos, color_mapping):
    plt.figure(figsize=(8, 8))

    # Assign colors based on the current assignment
    node_colors = ['#ffffff'] * len(G.nodes)  # Default is white for unassigned nodes
    for node, time_slot in assignment.items():
        node_colors[list(G.nodes).index(node)] = color_mapping[time_slot]  # Use valid HEX color

    # Draw the graph
    nx.draw(G, pos, with_labels=True, node_color=node_colors, font_weight='bold',
            node_size=2000, font_size=12, font_color='black', edge_color='black',
            linewidths=2, node_shape='o', edgecolors='black')

    plt.title(f"Exam Scheduling Step {step}")
    plt.show()

# Step 4: Check if the color (time slot) assignment is valid for a course
def is_valid_color(G, node, color, assignment):
    for neighbor in G.neighbors(node):
        if neighbor in assignment and assignment[neighbor] == color:
            return False
    return True

# Step 5: MRV Heuristic
def select_unassigned_node_MRV(G, assignment, colors):
    unassigned_nodes = [node for node in G.nodes if node not in assignment]
    mrv_node = None
    min_remaining_values = float('inf')

    for node in unassigned_nodes:
        remaining_values = sum(is_valid_color(G, node, color, assignment) for color in colors)
        if remaining_values < min_remaining_values:
            min_remaining_values = remaining_values
            mrv_node = node

    return mrv_node

# Step 6: Degree Heuristic
def select_unassigned_node_Degree(G, assignment):
    unassigned_nodes = [node for node in G.nodes if node not in assignment]
    degree_node = max(unassigned_nodes, key=lambda node: len([n for n in G.neighbors(node) if n not in assignment]))
    return degree_node

# Step 7: Backtracking with heuristics
def backtracking_with_heuristics(G, assignment, heuristic, step, colors, pos, color_mapping):
    if len(assignment) == len(G.nodes):
        return assignment  # All nodes are colored

    # Choose the next node to assign a time slot using the chosen heuristic
    if heuristic == 'MRV':
        node = select_unassigned_node_MRV(G, assignment, colors)
    elif heuristic == 'Degree':
        node = select_unassigned_node_Degree(G, assignment)
    else:
        node = list(G.nodes)[len(assignment)]

    for color in colors:
        if is_valid_color(G, node, color, assignment):
            assignment[node] = color  # Assign the time slot (color)
            visualize_graph(G, assignment, step, pos, color_mapping)  # Visualize the current step
            result = backtracking_with_heuristics(G, assignment, heuristic, step+1, colors, pos, color_mapping)
            if result:
                return result  # Return if a valid scheduling is found
            del assignment[node]  # Backtrack if no valid solution is found

    return None

# Step 8: Exam scheduling function with user input
def exam_scheduling():
    # Manually created course conflict graph
    G = create_exam_schedule_graph()
    colors, color_mapping = generate_colors(len(G.nodes))

    # Heuristic choice input with validation
    heuristic = None
    while heuristic not in ['1', '2']:
        print("Choose a heuristic:")
        print("1. Minimum Remaining Values (MRV)")
        print("2. Degree Heuristic")
        heuristic = input("Enter 1 or 2: ")

    if heuristic == '1':
        heuristic = 'MRV'
    elif heuristic == '2':
        heuristic = 'Degree'

    # Get the layout for graph visualization
    pos = nx.spring_layout(G, seed=42)  # Use seed for consistent layout
    final_assignment = backtracking_with_heuristics(G, {}, heuristic, 1, colors, pos, color_mapping)

    if final_assignment:
        print("Final Time Slot Assignment:", final_assignment)
    else:
        print("No valid assignment found.")

# Step 9: Run the exam scheduling
exam_scheduling()
